In [673]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [674]:
# Cargando el df

df = pd.read_csv('Churn_Modelling.csv')

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [675]:
print(f'El dataset cuenta con {df.shape[0]} filas y {df.shape[1]} columnas')

El dataset cuenta con 10000 filas y 14 columnas


# **PRE PROCESAMIENTO DE DATOS**

In [676]:
# Revisando si la cantidad de customers únicos es igual a la del dataframe
len(df['CustomerId'].unique())

10000

### Eliminando columnas irrelevantes

In [677]:
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)

#revisando la eliminación
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [678]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


- A primera vista no hay datos nulos
- Existen 2 columnas objects que son categoricas `Gender` y `Age`
- Todas las demás features son numéricas

In [679]:
print(f'Cantidad de datos duplicados {df.duplicated().sum()}') 

Cantidad de datos duplicados 0


In [680]:
categorical_columns = df.select_dtypes(include='object')
numeric_columns = df.select_dtypes(include=['int', 'float'])

for col in categorical_columns:
    
    print(df[col].value_counts())
    print('==='*60)


Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64
Gender
Male      5457
Female    4543
Name: count, dtype: int64


- No se observa problemas en cuanto a que existan categorias con problemas de escritura u otro tipo que no nos permitan hacer un label encoding

In [681]:
labeler = LabelEncoder()
df['Gender'] = labeler.fit_transform(df['Gender'])

In [682]:
onehot = OneHotEncoder(sparse_output=False)
encoder_geo = onehot.fit_transform(df[['Geography']])

In [683]:
column_names = onehot.get_feature_names_out(['Geography'])
column_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [684]:
df[column_names] = encoder_geo

In [685]:
df = df.drop(columns='Geography', axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [686]:
# Guarda objetos
def saving(path, object):
    with open(path, 'wb') as file:
        pickle.dump(object, file)
    return object

In [687]:
label_encoder_save = saving('label_encoder.pkl', labeler)
onehot_encoder_save = saving('onehot_encoder.pkl', onehot)

In [688]:
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns

umbral = 10 
vars_continuas = [col for col in numeric_columns if df[col].nunique() > umbral]

vars_discretas = [col for col in numeric_columns if df[col].nunique() <= umbral]

In [689]:
df[numeric_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
CreditScore,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
Gender,10000.0,0.545700,0.497932,0.00,0.00,1.000,1.0000,1.00
Age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48
Exited,10000.0,0.203700,0.402769,0.00,0.00,0.000,0.0000,1.00


In [690]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

## **ENTRENAMIENTO**

In [691]:
X = df.drop(columns='Exited', axis=1)
y= df['Exited']

In [692]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [693]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [694]:
scaler_filer = saving('scaler_features.pkl', scaler)

In [695]:
import tensorflow as tf

In [696]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [697]:
model = Sequential(
   [ Dense(64, activation='relu', input_shape=(X_train.shape[1], )),
    
    Dense(32, activation='relu'),

    Dense(1, activation='sigmoid')
]

)

model.summary()

/home/kevin-llanos/ann_project/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_103 (Dense)               │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [698]:
from tensorflow.keras.metrics import F1Score, Accuracy

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [699]:
os.makedirs('log/fit', exist_ok=True)
log_dir = "log/fit" + datetime.datetime.now().strftime('%d-%m-%Y - %H:%M:%S')

In [700]:
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [701]:
early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [702]:
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks= [tensorflow_callback, early]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7935 - loss: 0.4480 - val_accuracy: 0.8585 - val_loss: 0.3515
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8509 - loss: 0.3636 - val_accuracy: 0.8575 - val_loss: 0.3491
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8550 - loss: 0.3511 - val_accuracy: 0.8590 - val_loss: 0.3438
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8642 - loss: 0.3415 - val_accuracy: 0.8670 - val_loss: 0.3434
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8638 - loss: 0.3329 - val_accuracy: 0.8540 - val_loss: 0.3433
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8572 - loss: 0.3472 - val_accuracy: 0.8615 - val_loss: 0.3509
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8653 - loss: 0.3290 - val_accuracy: 0.8640 - val_loss: 0.3365
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8691 - loss: 0.3239 - val_accu

In [703]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

clasif = classification_report(y_pred_binary, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [704]:
print(clasif)

              precision    recall  f1-score   support

           0       0.97      0.88      0.92      1761
           1       0.46      0.78      0.58       239

    accuracy                           0.86      2000
   macro avg       0.71      0.83      0.75      2000
weighted avg       0.91      0.86      0.88      2000



In [761]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42,k_neighbors=2)
X_train_resample, y_train_resample = smote.fit_resample(X_train, y_train)

In [762]:
model2 = Sequential(
   [ Dense(64, activation='relu', input_shape=(X_train_resample.shape[1], )),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
]

)

model2.summary()

/home/kevin-llanos/ann_project/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_118 (Dense)               │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_120 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [763]:
from tensorflow.keras.metrics import F1Score, Accuracy

opt2 = tf.keras.optimizers.Adam(learning_rate=0.01)
model2.compile(loss='binary_crossentropy', optimizer=opt2, metrics=['accuracy'])

In [764]:
history2 = model2.fit(
    X_train_resample, y_train_resample,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks= [tensorflow_callback, early]
)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6825 - loss: 0.5914 - val_accuracy: 0.7865 - val_loss: 0.5033
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7512 - loss: 0.5115 - val_accuracy: 0.8220 - val_loss: 0.4552
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7582 - loss: 0.5095 - val_accuracy: 0.8120 - val_loss: 0.4730
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7689 - loss: 0.4896 - val_accuracy: 0.7505 - val_loss: 0.4896
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7718 - loss: 0.4913 - val_accuracy: 0.7970 - val_loss: 0.4467
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7773 - loss: 0.4751 - val_accuracy: 0.8165 - val_loss: 0.4620
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7741 - loss: 0.4810 - val_accuracy: 0.8185 - val_loss: 0.4263
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7752 - loss: 0.4686 - val_accu

In [773]:
from sklearn.metrics import classification_report

y_pred = model2.predict(X_test)
y_pred_binary = (y_pred > 0.7).astype(int)

clasif = classification_report(y_pred_binary, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [775]:
model.save('model.h5')